# Pre-process fight data

In [16]:
import pandas as pd
import numpy as np
df_fight = pd.read_csv('./data/data.csv')
df_fight_raw = pd.read_csv('./data/raw_total_fight_data.csv', sep=";")

### Separate red/blue fighter data

In [17]:
def create_single_fight_df(pfx, opp_pfx, color, df):
    cols_single_fight = [k for k in df.columns if k[:2] != opp_pfx or k == f'{opp_pfx}fighter']
    df_single = df.loc[:, cols_single_fight]
    df_single.columns = df_single.columns.str.removeprefix(pfx)
    df_single.columns = df_single.columns.str.replace(f'{opp_pfx}fighter', 'opponent')
    df_single['fighter_color'] = color
    df_single['outcome'] = np.where(df_single['Winner'] == color, "Won", "Lost")
    return df_single

In [18]:
df_red_fight = create_single_fight_df('R_', 'B_', 'Red', df_fight)
df_blue_fight = create_single_fight_df('B_', 'R_', 'Blue', df_fight)

### Combine fighter data and write to file

In [19]:
df_fight_by_fighters = pd.concat([df_red_fight, df_blue_fight], axis=0).sort_values('date')
df_fight_by_fighters = df_fight_by_fighters.sort_values(by=['fighter', 'date'])

In [20]:
df_fight_by_fighters.to_csv('../src/data/fight_by_fighters.csv')

### Create index of fighters based on those in data set

In [21]:
df_fighters_list = df_fight_by_fighters.copy().drop_duplicates(subset='fighter', keep="last")

In [22]:
df_fighters_list.to_csv('../src/data/fighters_list.csv')